In [1]:
import tensorflow as tf
import numpy as np;
import pandas as pd;

BATCH_SIZE=50;
LEARNING_RATE = 1e-4
image_size=784
label=10;
image_width=28;
image_height=28;

In [2]:
# weight initialization
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [3]:
LOAD_FILE=input("Enter the name of the model to load the tensorflow model");
test=pd.read_csv('~/Documents/GIT_HUB/MNIST/test.csv').values
test=test.astype(np.float);
test=np.multiply(test,1.0/255.0);
x = tf.placeholder('float', shape=[None, image_size])
#W=tf.placeholder('float',shape=[images.shape[1],10])
y_=tf.placeholder('float',shape=[None,label])
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

image = tf.reshape(x, [-1,image_width , image_height,1])


h_conv1 = tf.nn.relu(conv2d(image, W_conv1) + b_conv1)
#print (h_conv1.get_shape()) # => (40000, 28, 28, 32)
h_pool1 = max_pool_2x2(h_conv1)
#print (h_pool1.get_shape()) # => (40000, 14, 14, 32)



# second convolutional layer
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)



W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])

h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder('float')
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)


#this is the output layer
W_fc2 = weight_variable([1024, label])
b_fc2 = bias_variable([label])

y = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)
cross_entropy = -tf.reduce_sum(y_*tf.log(y))


# optimisation function
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))
train_step = tf.train.AdamOptimizer(LEARNING_RATE).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float'))
predict = tf.argmax(y,1)
sess = tf.InteractiveSession()
saver = tf.train.Saver()
init = tf.initialize_all_variables()

Enter the name of the model to load the tensorflow modelmodel.ckpt


In [5]:
try:
    with tf.Session() as sess:
        saver.restore(sess,LOAD_FILE)
        predicted_values=np.zeros(shape=(test.shape[0]));
        for i in range(0,test.shape[0]//BATCH_SIZE):
            predicted_values[i*(BATCH_SIZE):(i+1)*BATCH_SIZE]=predict.eval(feed_dict={x:test[(i*BATCH_SIZE):(i+1)*BATCH_SIZE],keep_prob:1.0});
        print(predicted_values[10])
    np.savetxt('submission.csv', 
               np.c_[range(1,len(test)+1),predicted_values], 
               delimiter=',', 
               header = 'ImageId,Label', 
               comments = '', 
               fmt='%d')
except OSError as e:
    print('FILE DO NOT EXIST')

7.0
